In [39]:
# Data manipulation
import pandas as pd
import numpy as np
# Evaluation of the model
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import roc_auc_score

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['font.size'] = 18
%matplotlib inline

MAX_EVALS = 5
import csv
from hyperopt import STATUS_OK
from timeit import default_timer as timer



DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [40]:
import json
import shlex
import subprocess
import logging
from pprint import pprint
import re
import os


In [41]:
def runthebenchmark(hyperparameters):
    os.chdir('/home/dsinghvi/project/progress/active/../')
    storeinfile(hyperparameters)
    out=subprocess.Popen(["python3","read_config.py"], shell=False, stdout=subprocess.PIPE)
    logging.basicConfig(level=logging.DEBUG)
    output=out.stdout.read().decode('utf-8')
    print(output)
    value = 312 #output.split(" ")[5]
    value = float(value)
    print(value)
    return float(1/value)
    #logging.basicConfig(filename='app.log', filemode='a', format='%(name)s - %(levelname)s - %(message)s',level=logging.DEBUG)


In [42]:
def storeinfile(hyperparameters):
    data={"mpi": {"romio_ds_read": "enable", "romio_ds_write": "disable"}, "lfs": {"setstripe": {"size": "16777216", "count": 4}}}
    data["lfs"]["setstripe"]["size"] = int(hyperparameters["setstripe-size"])
    data["lfs"]["setstripe"]["count"] = int(hyperparameters["setstripe-count"])

    data["mpi"]["romio_ds_read"] = hyperparameters["romio_ds_read"]
    data["mpi"]["romio_ds_write"] = hyperparameters["romio_ds_write"]

    with open("confex.json","w") as fp:
        json.dump(data,fp)
    print(data)


In [43]:
def objective(hyperparameters):
    global ITERATION
    ITERATION += 1
    start = timer()
    result = runthebenchmark(hyperparameters)
    run_time = timer() - start
    print({'result': result, 'hyperparameters': hyperparameters, 'iteration': ITERATION, 'iteration_time': run_time, 'status': STATUS_OK})
    return {'loss': float(result), 'hyperparameters': hyperparameters, 'iteration': ITERATION, 'iteration_time': run_time, 'status': STATUS_OK}
    

In [44]:
from hyperopt import hp
from hyperopt.pyll.stochastic import sample

space = {
    'romio_ds_read' : hp.choice('romio_ds_read',['enable','disable']),
    'romio_ds_write' : hp.choice('romio_ds_write',['enable','disable']),
    'setstripe-size' : 65536*(hp.qloguniform('setstripe-size',0,3,1)),
    'setstripe-count' : hp.quniform('setstripe-count',0,20,1)
}

In [45]:
x = sample(space)
print(x)

{'romio_ds_read': 'disable', 'romio_ds_write': 'disable', 'setstripe-count': 9.0, 'setstripe-size': 196608.0}


In [46]:
from hyperopt import tpe

# Create the algorithm
tpe_algorithm = tpe.suggest


from hyperopt import Trials

# Record results
trials = Trials()

from hyperopt import fmin


ITERATION = 0

best = fmin(fn = objective, space = space, algo = tpe.suggest, trials = trials, max_evals = MAX_EVALS)


  0%|          | 0/5 [00:00<?, ?it/s, best loss: ?]

INFO:hyperopt.tpe:tpe_transform took 0.005253 seconds
INFO:hyperopt.tpe:TPE using 0 trials


{'mpi': {'romio_ds_read': 'enable', 'romio_ds_write': 'disable'}, 'lfs': {'setstripe': {'size': 131072, 'count': 3}}}
S3D-IO 50-50-100-2-2-4-1 455.24 0.03 0.07 244.73 0.09 0.37 0.05 0.01 131072 3 enable disable 

312.0                                              
{'result': 0.003205128205128205, 'hyperparameters': {'romio_ds_read': 'enable', 'romio_ds_write': 'disable', 'setstripe-count': 3.0, 'setstripe-size': 131072.0}, 'iteration': 1, 'iteration_time': 4.27945130225271, 'status': 'ok'}
 20%|██        | 1/5 [00:04<00:17,  4.29s/it, best loss: 0.003205128205128205]

INFO:hyperopt.tpe:tpe_transform took 0.006875 seconds
INFO:hyperopt.tpe:TPE using 1/1 trials with best loss 0.003205


{'mpi': {'romio_ds_read': 'enable', 'romio_ds_write': 'disable'}, 'lfs': {'setstripe': {'size': 327680, 'count': 8}}}
S3D-IO 50-50-100-2-2-4-1 242.33 0.03 0.13 179.89 0.09 0.51 0.03 0.01 327680 8 enable disable 

312.0                                                                         
{'result': 0.003205128205128205, 'hyperparameters': {'romio_ds_read': 'enable', 'romio_ds_write': 'disable', 'setstripe-count': 8.0, 'setstripe-size': 327680.0}, 'iteration': 2, 'iteration_time': 4.302283903583884, 'status': 'ok'}
 40%|████      | 2/5 [00:08<00:12,  4.30s/it, best loss: 0.003205128205128205]

...

{'mpi': {'romio_ds_read': 'enable', 'romio_ds_write': 'disable'}, 'lfs': {'setstripe': {'size': 131072, 'count': 3}}}
S3D-IO 50-50-100-2-2-4-1 231.66 0.03 0.13 226.95 0.09 0.40 0.04 0.01 131072 3 enable disable 

312.0                                                                         
{'result': 0.003205128205128205, 'hyperparameters': {'romio_ds_read': 'enable', 'romio_ds_write': 'disable', 'setstripe-count': 3.0, 'setstripe-size': 131072.0}, 'iteration': 3, 'iteration_time': 4.22165332455188, 'status': 'ok'}
 60%|██████    | 3/5 [00:12<00:08,  4.28s/it, best loss: 0.003205128205128205]

INFO:hyperopt.tpe:tpe_transform took 0.005914 seconds
INFO:hyperopt.tpe:TPE using 3/3 trials with best loss 0.003205


{'mpi': {'romio_ds_read': 'disable', 'romio_ds_write': 'enable'}, 'lfs': {'setstripe': {'size': 393216, 'count': 6}}}
S3D-IO 50-50-100-2-2-4-1 289.43 0.03 0.11 102.41 0.09 0.89 0.03 0.01 393216 6 disable enable 

312.0                                                                         
{'result': 0.003205128205128205, 'hyperparameters': {'romio_ds_read': 'disable', 'romio_ds_write': 'enable', 'setstripe-count': 6.0, 'setstripe-size': 393216.0}, 'iteration': 4, 'iteration_time': 4.761988355778158, 'status': 'ok'}
 80%|████████  | 4/5 [00:17<00:04,  4.43s/it, best loss: 0.003205128205128205]

INFO:hyperopt.tpe:tpe_transform took 0.005851 seconds
INFO:hyperopt.tpe:TPE using 4/4 trials with best loss 0.003205


{'mpi': {'romio_ds_read': 'enable', 'romio_ds_write': 'disable'}, 'lfs': {'setstripe': {'size': 458752, 'count': 15}}}
S3D-IO 50-50-100-2-2-4-1 353.73 0.03 0.09 110.35 0.09 0.83 0.05 0.01 458752 15 enable disable 

312.0                                                                         
{'result': 0.003205128205128205, 'hyperparameters': {'romio_ds_read': 'enable', 'romio_ds_write': 'disable', 'setstripe-count': 15.0, 'setstripe-size': 458752.0}, 'iteration': 5, 'iteration_time': 4.559067235328257, 'status': 'ok'}
100%|██████████| 5/5 [00:22<00:00,  4.48s/it, best loss: 0.003205128205128205]


In [ ]:
###Loader
data = pd.read_csv('../stats.txt', delim_whitespace=True)
array = data.values

output_col = input("Number of output column indexed from 0")
output_col = 2#int(output_col)
(n,m) = array.shape
print(array)                                                                           
X = array[:,0:m]
Y = array[:,output_col]
Y=Y.astype('int')

X=np.delete(X,output_col,1)

k=0
for i in range(0,m-1):
    if type(X[0][k]) == str:
        X=np.delete(X,k,1)
    #    print(X)
    else : 
        k=k+1
print(X)
###Model
model1(X,Y)


